In [86]:
%load_ext autoreload
%autoreload 2

from DDR import process_ddr
import pandas as pd
import dill as pickle
from datetime import datetime
from data_prep import get_efd_rf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS\ddrunpacked_full2.csv"
capacity_df = pd.read_csv(archive)

,Time,airport,capacity
0,2023-06-23 09:15:00,LTBA,0.175000
1,2023-10-24 09:15:00,LTBA,0.175000
2,2023-09-03 13:45:00,LTBA,0.175000
3,2023-09-11 06:30:00,LPPT,0.151316
4,2023-08-03 18:45:00,LPPT,0.151316
...,...,...,...
2171087,2024-03-31 18:45:00,LFLL,0.000000
2171088,2024-03-31 19:00:00,LFLL,0.000000
2171089,2024-03-31 19:15:00,LFLL,0.000000
2171090,2024-03-31 19:30:00,LFLL,0.000000


In [ ]:
import itertools
archive= r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS"
# archive= r"C:\Users\iLabs_6\Documents\Tex\realtimetest\efd"
taf_archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS\taf_reports.pkl"
# taf_archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS226\taf22.pkl"
# reg_archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS226\regulations226"
# archive = r"/Users/tex/Documents/Studie/SAT/Thesis/Code-data/out22"

# taf_archive = r"/Users/tex/Documents/Studie/SAT/Thesis/Code-data/out22"
with open(taf_archive, 'rb') as f:
    taf_ar = pickle.load(f)
#

start_date = datetime(2024, 3, 1, 0, 0)
end_date = datetime(2024, 4, 1, 0, 0)
# start_date = datetime(2022, 6, 1, 0, 0)
# end_date = datetime(2022, 7, 1, 0, 0)

efd = get_efd_rf(archive, start_date, end_date, taf_reports=taf_ar, reg_reports=None)


Loading chunk 0 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_0.pkl...
Chunk 0 loaded and combined.
Loading chunk 1 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_1.pkl...
Chunk 1 loaded and combined.
Loading chunk 2 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_2.pkl...
Chunk 2 loaded and combined.
Loading chunk 3 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_3.pkl...
Chunk 3 loaded and combined.
Loading chunk 4 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_4.pkl...
Chunk 4 loaded and combined.
Loading chunk 5 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_5.pkl...
Chunk 5 loaded and combined.
Loading chunk 6 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_6.pkl...
Chunk 6 loaded and combined.
Loading chunk 7 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_7.pkl...
Chunk 7 loaded and combined.
Loading chunk 8 from C:\Users\iLabs_6\Documents\Tex\allwithCBAS\efd_8.pkl...
Chunk 8 loaded and combined.
Loading chunk 9 from C:\Users\iLabs_6\Document

In [ ]:
efd.keys()

In [ ]:
archive = r"C:\Users\iLabs_6\Documents\Tex\realtimetest\extended-atot-5min-ko2.csv"
try:
    extended_df = pd.read_csv(archive)
    extended_df['distance'] = extended_df['distance'].astype(float)
    extended_df['atfmdelay'] = extended_df['atfmdelay'].astype(float)
    cols = extended_df.columns.tolist()
    datetime_cols = ['ETOT', 'EOBT', 'ETA', 'cbasentry', 'TSAT', 'TOBT']
    timedelta_cols =[col for col in extended_df.columns if 'CBAS' in col]
    for col in datetime_cols:
        extended_df[col] = pd.to_datetime(extended_df[col], errors='coerce')
    for col in timedelta_cols:
        extended_df[col] = pd.to_timedelta(extended_df[col], errors='coerce')
 
except Exception as e:
    print(f'{e=}')
    extended_df = prepare_dataset_with_updates(efd, cap=capacity_df, horizon_hours=5, dt_minutes=5, taf_ar=taf_ar, save_ar= archive, mode='atot')
